This workbook is intent to assess and clean the full_train dataset, remove missing and invalid trip.

In [2]:
import pandas as pd
import numpy as np
from geopy.distance import great_circle
pd.set_option("max_colwidth",1000000)
pd.set_option('max_columns', 50)
import datetime


In [2]:
train=pd.read_csv("full_test_parsed.csv")

C:\Users\TANGP05\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625134 entries, 0 to 625133
Data columns (total 20 columns):
id                     625134 non-null object
vendor_id              625134 non-null int64
pickup_datetime        625134 non-null object
passenger_count        625134 non-null int64
pickup_longitude       625134 non-null float64
pickup_latitude        625134 non-null float64
dropoff_longitude      625134 non-null float64
dropoff_latitude       625134 non-null float64
store_and_fwd_flag     625134 non-null object
distance               625134 non-null float64
duration               625134 non-null float64
route_coordinates      625134 non-null object
pickup_date            625134 non-null object
date                   625134 non-null object
maximum temperature    625134 non-null int64
minimum temperature    625134 non-null int64
average temperature    625134 non-null float64
precipitation          625134 non-null object
snow fall              625134 non-null object
snow depth  

In [4]:
train['pickup_datetime'] = pd.to_datetime(train.pickup_datetime)
train['pickup_hour'] = train['pickup_datetime'].dt.hour
train['pickup_weekday'] = train['pickup_datetime'].dt.weekday
train['pickup_month']=train['pickup_datetime'].dt.month
train['snow depth']=train['snow depth'].apply(lambda x: float(x) if x!='T' else 0.001)
train['snow']=train['snow depth'].apply(lambda x: 1 if x>0.01 else 0)



def holiday(x):
    x1=x.date()
    if x1 in (datetime.date(2016,1,18),datetime.date(2016,2,15),datetime.date(2016,5,30),datetime.date(2016,5,28),datetime.date(2016,6,19)):
        return 1
    else:
        return 0
    
train['holiday']=train['pickup_datetime'].apply(lambda x: holiday(x)) 

In [5]:
train['great_circle_distance']=train.apply(lambda row: great_circle((row['pickup_latitude'],row['pickup_longitude']),(row['dropoff_latitude'],row['dropoff_longitude'])).meters,axis=1)

In [6]:
def hour_count(date,hour):
    return (date.date()-datetime.date(2016,1,1)).days*24+hour

train['hour_count']=train.apply(lambda row: hour_count(row['pickup_datetime'],row['pickup_hour']),axis=1)

In [7]:
feature=['id','vendor_id','pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude',
        'great_circle_distance','duration','hour_count','passenger_count','pickup_hour','pickup_weekday','pickup_month','snow','holiday',
        'distance','route_coordinates']
train[feature].to_csv("test_full_parsed_clean0.1.csv",index=False)

In [4]:
features_test=['id','vendor_id','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','hour_count',
        'great_circle_distance','duration','passenger_count','pickup_hour','pickup_weekday','pickup_month','snow','holiday',
        'distance','route_coordinates']
train=pd.read_csv("test_full_parsed_clean0.1.csv",usecols=features_test)

In [5]:
train.groupby('passenger_count').size()

passenger_count
0        23
1    443447
2     90027
3     25686
4     12017
5     33411
6     20521
9         2
dtype: int64

In [6]:
train['passenger_count'].loc[train['passenger_count'] == 9] = 7

C:\Users\TANGP05\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
def transformation(train,istrain):
    
    train['vendor_id'] = train['vendor_id'].map( {1: 0, 2: 1} ).astype(int)
    
    temp = pd.get_dummies(train['pickup_hour'],prefix='pickup_hour')
    train = pd.concat([train,temp],axis=1)
    train = train.drop(["pickup_hour"], axis=1)

    temp = pd.get_dummies(train['pickup_weekday'],prefix='pickup_weekday')
    train = pd.concat([train,temp],axis=1)
    train = train.drop(["pickup_weekday"], axis=1)

    temp = pd.get_dummies(train['pickup_month'],prefix='pickup_month')
    train = pd.concat([train,temp],axis=1)
    train = train.drop(["pickup_month"], axis=1)

    temp = pd.get_dummies(train['passenger_count'],prefix='passenger_count')
    train = pd.concat([train,temp],axis=1)
    train = train.drop(["passenger_count"], axis=1)
    
    train['log_duration']=np.log(train['duration'].values+1)
    train=train.drop(["duration"], axis=1)
    if istrain=="Y":
        train['log_trip_duration']=np.log(train['trip_duration'].values+1)
        train=train.drop(["trip_duration"], axis=1)
    
    return train

In [8]:
train=transformation(train,istrain="N")

In [9]:
train.shape

(625134, 58)

In [10]:
feature_out=['id','log_duration','pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'great_circle_distance','distance','hour_count',
       'snow', 'holiday', 'vendor_id', 'pickup_hour_0',
       'pickup_hour_1', 'pickup_hour_2', 'pickup_hour_3', 'pickup_hour_4',
       'pickup_hour_5', 'pickup_hour_6', 'pickup_hour_7', 'pickup_hour_8',
       'pickup_hour_9', 'pickup_hour_10', 'pickup_hour_11',
       'pickup_hour_12', 'pickup_hour_13', 'pickup_hour_14',
       'pickup_hour_15', 'pickup_hour_16', 'pickup_hour_17',
       'pickup_hour_18', 'pickup_hour_19', 'pickup_hour_20',
       'pickup_hour_21', 'pickup_hour_22', 'pickup_hour_23',
       'pickup_weekday_0', 'pickup_weekday_1', 'pickup_weekday_2',
       'pickup_weekday_3', 'pickup_weekday_4', 'pickup_weekday_5',
       'pickup_weekday_6', 'pickup_month_1', 'pickup_month_2',
       'pickup_month_3', 'pickup_month_4', 'pickup_month_5',
       'pickup_month_6', 'passenger_count_0', 'passenger_count_1',
       'passenger_count_2', 'passenger_count_3', 'passenger_count_4',
       'passenger_count_5', 'passenger_count_6', 'passenger_count_7',
       'route_coordinates']
train.to_csv("test_full_parsed_clean2.1.csv",index=False, columns=feature_out)

In [17]:
train.groupby('hour_count').size()

hour_count
0       213
1       282
2       229
3       216
4       118
5        90
6        48
7        56
8        49
9        72
10       80
11       99
12      113
13      154
14      160
15      157
16      159
17      140
18      146
19      112
20      115
21      132
22      130
23      111
24      112
25       75
26       66
27       49
28       53
29       19
       ... 
4338    194
4339    226
4340    189
4341    203
4342    192
4343    160
4344    103
4345     57
4346     37
4347     30
4348     29
4349     36
4350    123
4351    140
4352    159
4353    154
4354    141
4355    148
4356    153
4357    147
4358    168
4359    157
4360    128
4361    171
4362    208
4363    174
4364    187
4365    221
4366    199
4367    200
Length: 4356, dtype: int64